### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
# Look at column names; this will be helpful for the steps below

list(purchase_data.columns)

# 'Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'

['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']

## Player Count

* Display the total number of players


In [3]:
purchase_data_array = purchase_data["SN"].unique()  # output is numpy.ndarray
player_count = len(purchase_data_array)
player_df = pd.DataFrame([{"Total Players":player_count}])
display(player_df)

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
number_of_unique_items = len(purchase_data["Item Name"].unique())

average_price_PAT = purchase_data["Price"].mean()
average_price_PAT_formatted = "$" + str(round(average_price_PAT, 2))

number_of_purchases = len(purchase_data["Purchase ID"].unique())

total_revenue = purchase_data["Price"].sum()
total_revenue_formatted = "$" + str(total_revenue)

summary_df = pd.DataFrame({"Number of Unique Items":[number_of_unique_items], 
                "Average Price":[average_price_PAT_formatted], 
                "Number of Purchases":[number_of_purchases],
                "Total Revenue":[total_revenue_formatted]})

display(summary_df)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
purchase_data_unique = purchase_data.drop_duplicates(subset = 'SN')
total_count_GD = purchase_data_unique["Gender"].value_counts()
precentage_of_players_GD = round((purchase_data_unique["Gender"].value_counts(normalize = True)) * 100, 2)


# Reformat using list comprehension, then rebuild Series
precentage_of_players_GD_formatted = pd.Series(["$" + str(value) for value in precentage_of_players_GD],
                                    index = precentage_of_players_GD.index.to_list())

# Convert each Series to a DataFrame, then merge
gender_freqs_df = total_count_GD.to_frame("Total Count")
gender_rfreqs_df = precentage_of_players_GD_formatted.to_frame("Percentage of Players")

gender_summary = pd.merge(gender_freqs_df, gender_rfreqs_df, left_index = True, right_index = True)
display(gender_summary)

,Total Count,Percentage of Players
Male,484,$84.03
Female,81,$14.06
Other / Non-Disclosed,11,$1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
gendered = purchase_data.groupby(["Gender"])

## Create lists of summary statistics from gendered DataFrameGroupBy
## Format using list comprehensions, when required

# get purchase count
purchase_count_PAG = gendered["Purchase ID"].count()  # Series

# Pull gender index from purchase_count_PAG; will use this in DataFrame (below)
Gender_index = purchase_count_PAG.index.to_list()

# get average purchase price
average_purchase_price_PAG = gendered["Price"].mean()  # Series
average_purchase_price_PAG_formatted = ["${0:.2f}".format(values) for values in average_purchase_price_PAG] # Now a list

# get total purchase value
total_purchase_value = gendered["Price"].sum()  # Series
total_purchase_value_formatted = ["${0:.2f}".format(values) for values in total_purchase_value] # Now a list

# get average total purchase per person 
means_unique = total_purchase_value / total_count_GD  # divisor from above "Gender Demographics" cell
means_unique_formatted = ["${0:.2f}".format(values) for values in means_unique]  # Now a list

## Make DataFrame; set "Gender" as index
gendered_df = pd.DataFrame({"Gender":Gender_index,
                            "Purchase Count":purchase_count_PAG, 
                            "Average Purchase Price":average_purchase_price_PAG_formatted,
                            "Total Purchase Value":total_purchase_value_formatted,
                            "Avg Total Purchase per Person":means_unique_formatted}).set_index(["Gender"])
display(gendered_df)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Find min and max age
min_age = purchase_data["Age"].min()
max_age = purchase_data["Age"].max()

# Use specified limits for age bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]

# Copy purchase_data_unique from "Gender Demographics" cell; recall these were based on unique 'SN'
# This step is required because we don't want to count the same individual more than once in the age bins.
purchase_data_cut = purchase_data_unique.copy()

# Use pd.cut() to bin (using "bins", from above)
purchase_data_cut["Age Ranges"] = pd.cut(purchase_data_cut["Age"], bins, include_lowest = True, right = True, precision = 0)

# Group the rows that have the same "Age Range" and count the number in each bin
purchase_age_ranges_groupby = purchase_data_cut.groupby(["Age Ranges"])
total_count_AD = purchase_age_ranges_groupby["Purchase ID"].count()  # Note that total_count_AD variable (from above) is overwritten here

# Count the total number of individuals over all age groups; we need this to get the percentages
sum_total_count_AD = sum(total_count_AD)
percentage_of_players_AD = (total_count_AD / sum_total_count_AD) * 100
percentage_of_players_AD_formatted = percentage_of_players_AD.map("{:.2f}%".format)

age_ranges = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

age_demographics_df = pd.DataFrame(zip(total_count_AD, percentage_of_players_AD_formatted), 
                             columns = ["Total Count", "Percentage of Players"], 
                             index = age_ranges)

display(age_demographics_df)


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
## The key difference between this summary and that above is that we need all of the purchase, regardless of 'SN'
## But we need the unique count of individuals in each group (total_count, from above) to get the average purchase per person

purchase_data_all_cut = purchase_data.copy()

# Cut based on the age ranges, again, but use purchase data, rather than purchase_data_unique
# Then count up the number of purchases within each bin.
purchase_data_all_cut["Age Ranges"] = pd.cut(purchase_data_all_cut["Age"], bins, include_lowest = True, right = True, precision = 0)
age_range_groupby = purchase_data_all_cut.groupby(["Age Ranges"])
purchase_count_PAA = age_range_groupby["Purchase ID"].count()

# Get average puchase price in each bin
average_purchase_price_AD = age_range_groupby["Price"].mean()
average_purchase_price_AD_formatted = ["${0:.2f}".format(values) for values in average_purchase_price_AD]

# Get total purchase value in each bin
total_purchase_value = age_range_groupby["Price"].sum()
total_purchase_value_formatted = ["${:.2f}".format(values) for values in total_purchase_value]

# Get average total purchase price per person using total_count for unique 'SN' from cell above
average_purchase_price_unique = total_purchase_value / total_count_AD
average_purchase_price_unique_formatted = ["${:.2f}".format(values) for values in average_purchase_price_unique]

purchasing_age_df = pd.DataFrame(zip(age_ranges, purchase_count_PAA, average_purchase_price_AD_formatted, total_purchase_value_formatted, average_purchase_price_unique_formatted),
                                 columns = ["Age Ranges","Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"])

purchasing_age_df = purchasing_age_df.set_index(["Age Ranges"])
display(purchasing_age_df)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Group buyers by "SN", then sum "Price" and sort to find the top five; get their usernames ("SN")
buyers_grouped = purchase_data.groupby(["SN"])
total_purchase_value = buyers_grouped["Price"].sum().sort_values(ascending = False)[0:5]
total_value = total_purchase_value.sort_index()
top_buyers = total_purchase_value.index.to_list() # Get their usernames ("SN")

## My general strategy here was to get a Series or list for each column.
## I then rebuilt them into a DataFrame in the last several steps.

# Make copy of purchase_data DataFrame and subset using .loc with the names of the top buyers as the index
top_purchasers = purchase_data.copy()
top_purchasers = top_purchasers.set_index(["SN"])
top_purchasers = top_purchasers.loc[top_buyers, "Price"].to_frame()

# Extract Series of 'Price' for the top buyers
top_purchasers_prices = top_purchasers["Price"]
top_purchasers_list = top_purchasers.index.to_list() # Pull index back out as list

# Rebuild data frame with two columns: 'SN' and 'Price'
top_purchasers_temp = pd.DataFrame(zip(top_purchasers_list, top_purchasers_prices),
                                   columns = ["SN", "Price"])

# Now that the rows only represent the top purchasers, group (again) by 'SN'
top_purchasers_groupby = top_purchasers_temp.groupby(["SN"])

# Get mean and pull out index to use in zip; convert DataFrame to Series
average_purchase_price_TS = top_purchasers_groupby.mean()  # DataFrame 
top_purchasers_sorted_index = average_purchase_price_TS.index.to_list()
mean_top_purchasers = average_purchase_price_TS["Price"]  # Series

# Get purchase count; convert DataFrame to Series
purchase_count = top_purchasers_groupby.count()  # DataFrame
purchase_count = purchase_count["Price"]  # Series

# Now zip the Series/lists together
top_purchasers_df = pd.DataFrame(zip(top_purchasers_sorted_index, purchase_count, mean_top_purchasers, total_value),
                                 columns = ["SN", "Purchase Count", "Average Purchase Price", "Total Purchase Value"]).set_index(["SN"])

# Sort
top_purchasers_df = top_purchasers_df.sort_values("Total Purchase Value", ascending = False)

# Format columns
top_purchasers_df["Average Purchase Price"] = top_purchasers_df["Average Purchase Price"].map("${:.2f}".format)
top_purchasers_df["Total Purchase Value"] = top_purchasers_df["Total Purchase Value"].map("${:.2f}".format)

display(top_purchasers_df)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
purchase_data_items = purchase_data[["Item ID", "Item Name", "Price"]]
purchase_data_items_groupby = purchase_data_items.groupby(["Item ID", "Item Name"])

purchase_count_MPI = purchase_data_items_groupby.count()
total_purchase_value_MPI = purchase_data_items_groupby.sum()
average_item_price = purchase_data_items_groupby.mean()

summary_df = pd.concat([purchase_count_MPI, average_item_price, total_purchase_value_MPI], axis = 1)
summary_df.columns = ["Purchase Count", "Item Price", "Total Purchase Value"]
summary_df_sorted_PC = summary_df.sort_values("Purchase Count", ascending = False)

summary_df_sorted_PC["Item Price"] = summary_df_sorted_PC["Item Price"].map("${0:.2f}".format)
summary_df_sorted_PC["Total Purchase Value"] = summary_df_sorted_PC["Total Purchase Value"].map("${0:.2f}".format)

summary_df_sorted_PC.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [14]:
summary_df_sorted_TPV = summary_df.sort_values("Total Purchase Value", ascending = False)

summary_df_sorted_TPV["Item Price"] = summary_df_sorted_TPV["Item Price"].map("${0:.2f}".format)
summary_df_sorted_TPV["Total Purchase Value"] = summary_df_sorted_TPV["Total Purchase Value"].map("${0:.2f}".format)

summary_df_sorted_TPV.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


# Analysis

*  Many different types of items were sold (= 179 unique items). Nevertheless, a few items accounted for a disproportionate number of purchases and total purchase value: Final Critic, Oathbreaker, and Fiery Glass Crusader. 


*  The average purhase price was around three dollars (\$3.05\). Overall revenue was seemingly low: \\$2.379.72. While the average purchase price was around three dollars, some individuals spent considerably more money, on average (max was \\$18.96). Hence averages may not be a totally reliable summary statistic since the distribution of average purchase prices was right skewed, albeit the fact that some users spent considerably more money on average is an important fact. 


*  Most purchasers were males 84\%. However the average purchase price and average total purchase price per person was higher for females than males (i.e., \\$3.20 vs. \\$3.02 and \\$4.47 vs. \\$4.07, respectively. The next step would be to test to determine if this is statistically significant (and not just a fluke of sampling error).


*  Most buyers were between 20 and 24 years old, with 80\% being between 15 and 29 years old. However, the youngest and oldest age brackets had higher average purchase prices and average total purchase prices per person. This also needs to be tested to determine if the difference is statistically significant.